In [1]:
from pathlib import Path
from sqlalchemy import create_engine
import pandas as pd
import sqlite3
import json
from census import Census

In [2]:
# Read the Net_generation_for_all_sectors_monthly CSV file 
data = pd.read_csv('Net_generation_for_all_sectors_.csv', sep=',', skiprows=4)

# Read the renewable_energy_consumption_source CSV file
renewable_energy_consumption = pd.read_csv('renewable_energy_consumption_source.csv')

transposed_data = data.transpose()

# Rename columns with values from the description row
transposed_data.columns = transposed_data.iloc[0]

# Drop the row after setting it as the header (optional)
transposed_data = transposed_data.drop('description')

# Delete 'Net generation for all sectors' and 'United States' columns
del transposed_data['Net generation for all sectors']
del transposed_data['United States']

# Display cleaned_data DataFrame the modified data
transposed_data.head()

description,United States : all fuels (utility-scale),United States : coal,United States : petroleum liquids,United States : petroleum coke,United States : natural gas,United States : other gases,United States : nuclear,United States : conventional hydroelectric,United States : other renewables,United States : wind,United States : all utility-scale solar,United States : geothermal,United States : biomass,United States : wood and wood-derived fuels,United States : other biomass,United States : hydro-electric pumped storage,United States : other,United States : all solar,United States : small-scale solar photovoltaic,United States : all utility-scale solar
units,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,NaN,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours
source key,ELEC.GEN.ALL-US-99.M,ELEC.GEN.COW-US-99.M,ELEC.GEN.PEL-US-99.M,ELEC.GEN.PC-US-99.M,ELEC.GEN.NG-US-99.M,ELEC.GEN.OOG-US-99.M,ELEC.GEN.NUC-US-99.M,ELEC.GEN.HYC-US-99.M,ELEC.GEN.AOR-US-99.M,ELEC.GEN.WND-US-99.M,ELEC.GEN.SUN-US-99.M,ELEC.GEN.GEO-US-99.M,ELEC.GEN.BIO-US-99.M,ELEC.GEN.WWW-US-99.M,ELEC.GEN.WAS-US-99.M,ELEC.GEN.HPS-US-99.M,ELEC.GEN.OTH-US-99.M,ELEC.GEN.TSN-US-99.M,ELEC.GEN.DPV-US-99.M,ELEC.GEN.SUN-US-99.M
Jan 2010,360957,173320,3187,1161,74173,909,72569,22383,NaN,6854,10,1312,4629,3126,1503,-565,1014,--,--,10
Feb 2010,319735,153044,1251,1122,66198,825,65245,20590,NaN,5432,33,1159,4277,2895,1382,-351,909,--,--,33
Mar 2010,312168,144406,1272,1198,63431,1010,64635,20886,NaN,8589,76,1307,4682,3090,1592,-325,1002,--,--,76


In [4]:
renewable_energy_consumption.head()

,MSN,YYYYMM,Value,Column_Order,Description,Unit
0,WDPRBUS,194913,1549.262,1,Wood Energy Production,Trillion Btu
1,WDPRBUS,195013,1562.307,1,Wood Energy Production,Trillion Btu
2,WDPRBUS,195113,1534.669,1,Wood Energy Production,Trillion Btu
3,WDPRBUS,195213,1474.369,1,Wood Energy Production,Trillion Btu
4,WDPRBUS,195313,1418.601,1,Wood Energy Production,Trillion Btu


In [5]:
# Convert 'YYYYMM' column to string
renewable_energy_consumption['YYYYMM'] = renewable_energy_consumption['YYYYMM'].astype(str)

# Filter out rows where 'YYYYMM' ends with '13'
renewable_energy_consumption_df = renewable_energy_consumption[~renewable_energy_consumption['YYYYMM'].str.endswith('13')]

# Change 'YYYYMM' column name to 'Date'
renewable_energy_consumption_df.rename(columns={'YYYYMM':'Date'}, inplace=True)

# Convert the 'Date' column to a datetime format
renewable_energy_consumption_df['Date'] = pd.to_datetime(renewable_energy_consumption_df['Date'].astype(str), format='%Y%m' )
# df['date_column'] = pd.to_datetime(df['date_column'].astype(str), format='%Y%m%d')


renewable_energy_consumption_df.head()

/var/folders/qh/b72b_cj57xq1mf0p07xjw8j40000gn/T/ipykernel_91708/636604446.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renewable_energy_consumption_df.rename(columns={'YYYYMM':'Date'}, inplace=True)
/var/folders/qh/b72b_cj57xq1mf0p07xjw8j40000gn/T/ipykernel_91708/636604446.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renewable_energy_consumption_df['Date'] = pd.to_datetime(renewable_energy_consumption_df['Date'].astype(str), format='%Y%m' )


,MSN,Date,Value,Column_Order,Description,Unit
24,WDPRBUS,1973-01-01,129.63,1,Wood Energy Production,Trillion Btu
25,WDPRBUS,1973-02-01,117.194,1,Wood Energy Production,Trillion Btu
26,WDPRBUS,1973-03-01,129.763,1,Wood Energy Production,Trillion Btu
27,WDPRBUS,1973-04-01,125.462,1,Wood Energy Production,Trillion Btu
28,WDPRBUS,1973-05-01,129.624,1,Wood Energy Production,Trillion Btu


In [6]:
renewable_energy_consumption_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7956 entries, 24 to 8929
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   MSN           7956 non-null   object        
 1   Date          7956 non-null   datetime64[ns]
 2   Value         7956 non-null   object        
 3   Column_Order  7956 non-null   int64         
 4   Description   7956 non-null   object        
 5   Unit          7956 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 435.1+ KB


In [7]:
# Filter data for all dates from year >= 2018
filtered_df = renewable_energy_consumption_df[(renewable_energy_consumption_df['Date'].dt.year >= 2010) & 
                                              (renewable_energy_consumption_df['Date'].dt.year <= 2020)]
filtered_df

,MSN,Date,Value,Column_Order,Description,Unit
505,WDPRBUS,2010-01-01,187.178,1,Wood Energy Production,Trillion Btu
506,WDPRBUS,2010-02-01,171.797,1,Wood Energy Production,Trillion Btu
507,WDPRBUS,2010-03-01,187.22,1,Wood Energy Production,Trillion Btu
508,WDPRBUS,2010-04-01,179.128,1,Wood Energy Production,Trillion Btu
509,WDPRBUS,2010-05-01,181.788,1,Wood Energy Production,Trillion Btu
...,...,...,...,...,...,...
8886,RETCBUS,2020-08-01,607.82,13,Total Renewable Energy Consumption,Trillion Btu
8887,RETCBUS,2020-09-01,573.79,13,Total Renewable Energy Consumption,Trillion Btu
8888,RETCBUS,2020-10-01,592.81,13,Total Renewable Energy Consumption,Trillion Btu
8889,RETCBUS,2020-11-01,612.464,13,Total Renewable Energy Consumption,Trillion Btu
